# Customer Lifetime Value
---
- Bir müşterinin bir şirketle kurduğu ilişki-iletişim süresince bu şirkete kazandıracağı parasal değerdir.
- Müşterilerle olan ilişkimizi düzenleyebilir, şirketimizde değer odaklı bir yaklaşım sergileyebiliriz.
- Pazarlama faaliyetlerine ayrılacak olan bütçenin belirlenmesine katkı sağlayacaktır.
---
- **Nasıl Hesaplanır?**


- **CLTV**: (Customer Value / Churn Rate) x Profit Margin
- **Customer Value**: Average Order Value x Purchase Frequency
- **Average Order Value**: Total Price / Total Transaction
- **Purchase Frequency**: Total Transaction / Total Number of Customers
- **Churn Rate**: 1 - Repeat Rate
- **Repeat Rate**: 1'den fazla alışveriş yapan müşteri sayısı / tüm müşteriler
- **Profit Margin**: Total Price x 0.10
    - 0.10 şirketler tarafından belirlenen bir değerdir.
---
- Sonuç olarak; her bir müşteri için hesaplanacak olan **CLTV** değerlerine göre bir sıralama yapıldığında, *CLTV* değerlerine göre belirli noktalardan bölme işlemi yaparak gruplar oluşturulduğunda müşterilerimiz segmentlere ayrılmış olacaktır.

### Uçtan Uca Müşteri Yaşam Boyu Değeri Hesaplama
---

1. Veri Hazırlama
2. Average Order Value (average_order_value = total_price / total_transaction)
3. Purchase Frequency (total_transaction / total_number_of_customers)
4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
5. Profit Margin (profit_margin =  total_price * 0.10)
6. Customer Value (customer_value = average_order_value * purchase_frequency)
7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
8. Segmentlerin Oluşturulması
9. Tüm İşlemlerin Fonksiyonlaştırılması

#### 1. Veriyi Hazırlama

- Veri Seti Hikayesi
- https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

- Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

- Değişkenler


- **InvoiceNo**: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
- **StockCode**: Ürün kodu. Her bir ürün için eşsiz numara.
- **Description**: Ürün ismi
- **Quantity**: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
- **InvoiceDate**: Fatura tarihi ve zamanı.
- **UnitPrice**: Ürün fiyatı (Sterlin cinsinden)
- **CustomerID**: Eşsiz müşteri numarası
- **Country**: Ülke ismi. Müşterinin yaşadığı ülke.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# .py script için çıktı görünümü ayarları
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df_ = pd.read_excel("../datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()

- *df_.copy()* kullanmamın nedeni veri seti boyutu büyük olduğundan dolayı veri okuma işlemi uzun sürebilmektedir. çalışmamın ilerleyen zamanlarında tekrardan veriyi okutma ihtiyacı duyarsam bu süreyi optimize etmek, beklememek adına kullanıyorum.

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [4]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

- veri setinin daha ölçülebilir olmasını istediğimizden eksik değerleri veri setinden çıkaracağız.
- iadeler veri setinin yapısını bozduğu için veri setinden çıkartmamız gerekiyor.

In [5]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,515255.00000,3.95637,127.68856,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


- eski ürün olamayacağından dolayı veri setini *Quantity*'e göre filtrelememiz gerekiyor.

In [7]:
df = df[df["Quantity"] > 0]

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,513134.00000,11.71543,92.97472,1.00000,1.00000,3.00000,10.00000,19152.00000
Price,513134.00000,3.97273,127.95193,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [9]:
df.dropna(inplace=True) # eksik değerleri veri kümesinden çıkarıyorum.

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.00000,13.58669,96.84223,1.00000,2.00000,5.00000,12.00000,19152.00000
Price,407695.00000,3.29419,34.75666,0.00000,1.25000,1.95000,3.75000,10953.50000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [11]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

- artık stok kodunu ve birim fiyatını bildiğim bir satın alma işlemi için toplam ne kadar bedel ödendiğini biliyorum. CLTV hesaplamasında metrikler için kullanacağız.

In [12]:
cltv_c = df.groupby("Customer ID").agg({"Invoice": lambda x: x.nunique(),
                                       "Quantity": lambda x: x.sum(),
                                       "TotalPrice": lambda x: x.sum()})

- **Invoice**'e uygulanan agg; böylece her bir müşterinin eşsiz kaç tane faturası olduğunu görüyoruz. bu şekilde *Total Transaction* değerine erişmiş oluyoruz.
- **Quantity** sadece analiz yapmak için kullandığımız birinci önceliğimiz olmayan bir agg.

In [13]:
cltv_c.head()

,Invoice,Quantity,TotalPrice
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


- analizin daha sağlıklı ve anlaşılır olabilmesi adına değişkenlerin isimlerini değiştiriyorum.

In [14]:
cltv_c.columns = ["total_transaction", "total_unit", "total_price"]

In [15]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [17]:
cltv_c.shape

(4314, 3)

- görüldüğü üzere müşterileri eşsizleştirdik. 4314 adet eşsiz fatura var. çoklamanın önüne geçmiş olduk.

#### 2. Average Order Value 
- (average_order_value = total_price / total_transaction)

In [18]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [19]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

In [20]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


#### 3. Purchase Frequency
- (total_transaction / total_number_of_customers)

In [21]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

- dataframe'in satırları müşterileri temsil ettiğinden dolayı *shape[0]* ile istediğimiz değerli elde etmiş oluyoruz.

In [22]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12348.00000,1,373,222.16000,222.16000,0.00023
12349.00000,3,993,2671.14000,890.38000,0.00070
12351.00000,1,261,300.93000,300.93000,0.00023


#### 4. Repeat Rate & Churn Rate
- (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)

In [23]:
cltv_c[cltv_c["total_transaction"] > 1].shape[0]
# birden fazla alışveriş yapan müşteri sayısı

2893

In [25]:
# repeat_rate
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

In [28]:
churn_rate = 1 - repeat_rate
churn_rate

0.3293926750115902

#### 5. Profit Margin
- (profit_margin = total_price * 0.10)

In [29]:
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10

In [30]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300


#### 6. Customer Value
- (customer_value = average_order_value * purchase_frequency)

In [31]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

In [32]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976


#### 7. Customer Lifetime Value
- (CLTV = (customer_value / churn_rate) x profit_margin)

In [33]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [34]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


- bu şekilde müşterileri kendi aralarında CLTV değerlerini hesapladık.
- bir sıralama yapıp bizim için en değerli müşterileri görmek istersek;

In [35]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [36]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
total_unit,4314.00000,1284.01113,6458.45205,1.00000,158.00000,382.00000,995.25000,220600.00000
total_price,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000
average_order_value,4314.00000,378.14723,492.51721,0.00000,181.95632,286.94600,423.53375,11880.84000
purchase_frequency,4314.00000,0.00103,0.00189,0.00023,0.00023,0.00046,0.00116,0.04752
profit_margin,4314.00000,204.72887,891.25232,0.00000,30.79500,70.55500,172.28025,34916.43500
customer_value,4314.00000,0.47457,2.06595,0.00000,0.07138,0.16355,0.39935,80.93749
cltv,4314.00000,5883.60651,156068.63594,0.00000,6.67369,35.03173,208.87042,8579573.77276


#### 8. Segmentlerin Oluşturulması

In [37]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


In [38]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

In [39]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A


- oluşturulan segmentleri analiz etmek için;

In [40]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

total_transaction                total_unit                   \
                      sum count     mean        sum count       mean   
segment                                                                
D                    1326  1079  1.22892     117616  1079  109.00463   
C                    2160  1078  2.00371     305135  1078  283.05659   
B                    4063  1078  3.76902     733211  1078  680.15863   
A                   11666  1079 10.81186    4383262  1079 4062.33735   

          total_price                  average_order_value                  \
                  sum count       mean                 sum count      mean   
segment                                                                      
D        192265.13000  1079  178.18826        169805.46550  1079 157.37300   
C        513016.45300  1078  475.89652        317422.35514  1078 294.45487   
B       1219605.20000  1078 1131.35918        420127.12365  1078 389.72832   
A       6907116.49100  1079 6401.40546        723972.22347  1079 670.96592   

        purchase_frequency               profit_margin                  \
                       sum count    mean           sum count      mean   
segment                                                                  
D                  0.30737  1079 0.00028   19226.51300  1079  17.81883   
C                  0.50070  1078 0.00046   51301.64530  1078  47.58965   
B                  0.94182  1078 0.00087  121960.52000  1078 113.13592   
A                  2.70422  1079 0.00251  690711.64910  1079 640.14055   

        customer_value                         cltv                    
                   sum count    mean            sum count        mean  
segment                                                                
D             44.56772  1079 0.04130     2849.32790  1079     2.64071  
C            118.91897  1078 0.11031    18184.26342  1078    16.86852  
B            282.70867  1078 0.26225   103549.86329  1078    96.05739  
A           1601.09330  1079 1.48387 25257295.04118  1079 23408.05843

In [41]:
cltv_c.to_csv("cltc_c.csv")

#### 9. Tüm İşlemlerin Fonksiyonlaştırılması

In [42]:
def create_cltv_c(dataframe, profit=0.10):

    # Veriyi hazırlama
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c